# Expected Goals

The concept of this app is that people can map where a shot for goal has been taken place and the app provides the live expected value of that shot

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import simpledialog
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import json
from pandas.io.json import json_normalize
import seaborn as sns
from matplotlib.patches import Arc, Rectangle, ConnectionPatch
from matplotlib.offsetbox import  OffsetImage
from matplotlib.patches import Arc
from sklearn import metrics
from functools import reduce

In [31]:
pd.set_option('display.max_rows', 400)

In [32]:
%matplotlib notebook

First I need to read in the data I have collected to date to feed my model

In [33]:
folder_path = '~/Documents/GAA Analytics/2023/'
folder_path1 = '~/Documents/GAA Analytics/2024/'

In [34]:
df = pd.read_excel(folder_path + "Football_Championship_2023_Shots.xlsx")
df1 = pd.read_excel(folder_path1 + "Football_Championship_2024_Shots.xlsx")

In [35]:
frames = [df, df1]
df = pd.concat(frames)

In [36]:
df2 = df.copy()

## Expected Goals

### Isolate Goals

In [37]:
df2 = df2[df2['Action'].str.contains('goal') | (df2['Action'].str.contains('pen miss'))]


### Add Value to scores

In [38]:
df2['Score'] = np.where((df2['Outcome'] == 'Score'), 1, 0)

### Preferred side

In [39]:
def is_preferable_side(y, foot):
    preferable_side = 0
    side = 'center'
    if (y<44):
        side = 'left'
    elif (y>44):
        side = 'right'
        
    if ((side=='left') & (foot=='right')) | ((side=='right') & (foot=='left')) | ((side=='right') & (foot=='hand')) | ((side=='left') & (foot=='hand')):
        preferable_side = 1
    
    return preferable_side

In [40]:
df2['Preferred_Side'] = df2.apply(lambda row:is_preferable_side(row['stand_y'], row['Foot']), axis=1)

### Add Values to Variables

In [41]:
df2['Pressure_Value'] = df2['Pressure'].replace({'y': 1, 'n': 0})
df2['Pressure_Count'] = df2['Pressure_Value'].apply(lambda x: 1 if x > 0 else 0)
df2['Position_Value'] = df2['Position'].replace({'goalkeeper': 0, 'back': 1, 'midfielder': 2, 'forward': 3})
df2['Foot_Value'] = df2['Foot'].replace({'right': 0, 'left': 1, 'hand': 2})

### Calculate the Shot Angle

In [42]:
for index, row in df2.iterrows():
    shot_x, shot_y = row['stand_x'], row['stand_y']
    goal_x, goal_y = 145, 44  # Assuming constant goal coordinates
    
    # Calculate the differences in x and y coordinates
    delta_x = goal_x - shot_x
    delta_y = goal_y - shot_y
    
    # Calculate the angle using arctan2 function
    angle_radians = np.arctan2(delta_y, delta_x)
    
    # Convert angle from radians to degrees
    angle_degrees = np.degrees(angle_radians)
    
    # Store the angle in the DataFrame
    df2.at[index, 'Shot_Angle'] = angle_degrees

### Placed Balls

In [43]:
df2.groupby("Action").count()

,Unnamed: 0,Date,Game,TeamName,Opposition,Half,Minute,Outcome,PlayerName,PlayerNumber,...,y,stand_x,stand_y,Score,Preferred_Side,Pressure_Value,Pressure_Count,Position_Value,Foot_Value,Shot_Angle
Action,,,,,,,,,,,,,,,,,,,,,
goal,60,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
goal miss,83,83,83,83,83,83,83,83,83,83,...,83,83,83,83,83,83,83,83,83,83
pen miss,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
penalty goal,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [44]:
df2['Placed_Ball'] = np.where((df2['Action'] == 'goal') | (df2['Action'] == 'goal miss'), 0, 1)


## Using Logistic Regression to find xGoal

In [45]:
X_cols = ['Preferred_Side', 'Pressure_Value', 'Position_Value',
       'Foot_Value', 'Shot_Angle', 'Shot_Distance', 'Placed_Ball']

X = df2[X_cols]
y = df2['Score']

adv_model = LogisticRegression()
adv_model.fit(X, y)
y_pred = adv_model.predict_proba(X)[:, 1]
metrics.r2_score(y, y_pred)

/Users/cathalmcconnell/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.22488567218998012

In [46]:
df2['xGoal'] = y_pred

In [47]:
df2

,Unnamed: 0,Date,Game,TeamName,Opposition,Half,Minute,Action,Outcome,PlayerName,...,stand_y,Score,Preferred_Side,Pressure_Value,Pressure_Count,Position_Value,Foot_Value,Shot_Angle,Placed_Ball,xGoal
5,52,2023-05-07,Connaught Final,Galway,Sligo,1st,10,goal,Score,Matthew Tierney,...,45.712707,1,1,0,0,3,1,-13.412134,0,0.678808
13,60,2023-05-07,Connaught Final,Galway,Sligo,1st,23,goal,Score,Matthew Tierney,...,40.222742,1,0,1,1,3,1,71.695057,0,0.599752
58,105,2023-05-07,Munster Final,Kerry,Clare,1st,16,goal,Score,Tony Brosnan,...,52.243108,1,1,0,0,3,1,-56.589577,0,0.621423
61,108,2023-05-07,Munster Final,Kerry,Clare,1st,19,goal,Score,Dara Moynihan,...,46.044761,1,0,0,0,3,0,-52.941903,0,0.700198
74,121,2023-05-07,Munster Final,Kerry,Clare,1st,32,goal,Score,David Clifford,...,46.398952,1,1,0,0,3,1,-24.504258,0,0.751584
78,125,2023-05-07,Munster Final,Clare,Kerry,2nd,38,goal miss,Miss,Ronan Lanigan,...,36.924336,0,1,1,1,3,2,65.873479,0,0.388935
81,129,2023-05-07,Munster Final,Kerry,Clare,2nd,41,goal,Score,David Clifford,...,39.387352,1,0,0,0,3,1,20.741607,0,0.439976
84,130,2023-05-07,Munster Final,Clare,Kerry,2nd,43,goal miss,Miss,Padraic Collins,...,34.976284,0,1,1,1,3,0,63.165652,0,0.261167
85,132,2023-05-07,Munster Final,Kerry,Clare,2nd,45,goal,Score,Paudie Clifford,...,53.023716,1,1,0,0,3,1,-58.409324,0,0.591117
120,529,2023-05-14,Leinster Final,Dublin,Louth,1st,23,goal,Score,Paul Mannion,...,40.289153,1,1,0,0,3,2,41.353608,0,0.707232


## Correlations

In [48]:
X_cols = ['Preferred_Side', 'Pressure_Value', 'Position_Value', 'Foot_Value', 'Shot_Angle', 'Shot_Distance', 'Placed_Ball']

# Calculate the correlation matrix
correlation_matrix = df2[X_cols + ['Score']].corr()

# Extract the correlation values for Score
score_correlation = correlation_matrix['Score'].sort_values()

# Print the correlation values
print(score_correlation)


Shot_Distance    -0.297797
Pressure_Value   -0.187267
Shot_Angle       -0.116868
Preferred_Side   -0.048378
Position_Value   -0.037646
Foot_Value        0.180791
Placed_Ball       0.185928
Score             1.000000
Name: Score, dtype: float64


In [49]:
X_cols = ['Preferred_Side', 'Pressure_Value', 'Position_Value', 'Foot_Value', 'Shot_Angle', 'Shot_Distance', 'Placed_Ball']

# Calculate the correlation matrix
correlation_matrix = df2[X_cols + ['xGoal']].corr()

# Extract the correlation values for Score
score_correlation = correlation_matrix['xGoal'].sort_values()



# Print the correlation values
print(score_correlation)


Shot_Distance    -0.741106
Pressure_Value   -0.412535
Shot_Angle       -0.290487
Preferred_Side   -0.100181
Position_Value   -0.088545
Placed_Ball       0.330573
Foot_Value        0.451053
xGoal             1.000000
Name: xGoal, dtype: float64


In [50]:
df2['xG_Value'] = df2['xGoal'] * 3

In [51]:
df2

,Unnamed: 0,Date,Game,TeamName,Opposition,Half,Minute,Action,Outcome,PlayerName,...,Score,Preferred_Side,Pressure_Value,Pressure_Count,Position_Value,Foot_Value,Shot_Angle,Placed_Ball,xGoal,xG_Value
5,52,2023-05-07,Connaught Final,Galway,Sligo,1st,10,goal,Score,Matthew Tierney,...,1,1,0,0,3,1,-13.412134,0,0.678808,2.036424
13,60,2023-05-07,Connaught Final,Galway,Sligo,1st,23,goal,Score,Matthew Tierney,...,1,0,1,1,3,1,71.695057,0,0.599752,1.799257
58,105,2023-05-07,Munster Final,Kerry,Clare,1st,16,goal,Score,Tony Brosnan,...,1,1,0,0,3,1,-56.589577,0,0.621423,1.864269
61,108,2023-05-07,Munster Final,Kerry,Clare,1st,19,goal,Score,Dara Moynihan,...,1,0,0,0,3,0,-52.941903,0,0.700198,2.100593
74,121,2023-05-07,Munster Final,Kerry,Clare,1st,32,goal,Score,David Clifford,...,1,1,0,0,3,1,-24.504258,0,0.751584,2.254753
78,125,2023-05-07,Munster Final,Clare,Kerry,2nd,38,goal miss,Miss,Ronan Lanigan,...,0,1,1,1,3,2,65.873479,0,0.388935,1.166806
81,129,2023-05-07,Munster Final,Kerry,Clare,2nd,41,goal,Score,David Clifford,...,1,0,0,0,3,1,20.741607,0,0.439976,1.319929
84,130,2023-05-07,Munster Final,Clare,Kerry,2nd,43,goal miss,Miss,Padraic Collins,...,0,1,1,1,3,0,63.165652,0,0.261167,0.783501
85,132,2023-05-07,Munster Final,Kerry,Clare,2nd,45,goal,Score,Paudie Clifford,...,1,1,0,0,3,1,-58.409324,0,0.591117,1.773351
120,529,2023-05-14,Leinster Final,Dublin,Louth,1st,23,goal,Score,Paul Mannion,...,1,1,0,0,3,2,41.353608,0,0.707232,2.121696


### Save xG file

In [55]:
df2.to_excel(folder_path1 + "Football_Championship_Combined_xGoal.xlsx")

In [56]:
df2.head()

,Unnamed: 0,Date,Game,TeamName,Opposition,Half,Minute,Action,Outcome,PlayerName,...,Score,Preferred_Side,Pressure_Value,Pressure_Count,Position_Value,Foot_Value,Shot_Angle,Placed_Ball,xGoal,xG_Value
5,52,2023-05-07,Connaught Final,Galway,Sligo,1st,10,goal,Score,Matthew Tierney,...,1,1,0,0,3,1,-13.412134,0,0.678808,2.036424
13,60,2023-05-07,Connaught Final,Galway,Sligo,1st,23,goal,Score,Matthew Tierney,...,1,0,1,1,3,1,71.695057,0,0.599752,1.799257
58,105,2023-05-07,Munster Final,Kerry,Clare,1st,16,goal,Score,Tony Brosnan,...,1,1,0,0,3,1,-56.589577,0,0.621423,1.864269
61,108,2023-05-07,Munster Final,Kerry,Clare,1st,19,goal,Score,Dara Moynihan,...,1,0,0,0,3,0,-52.941903,0,0.700198,2.100593
74,121,2023-05-07,Munster Final,Kerry,Clare,1st,32,goal,Score,David Clifford,...,1,1,0,0,3,1,-24.504258,0,0.751584,2.254753
